In [1]:
from importlib import reload

from pyspark.sql.functions import col
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

from hhop import get_spark_builder
spark_builder = get_spark_builder('custom_name'); 
spark = spark_builder.getOrCreate(); sc = spark.sparkContext; sc.setLogLevel("ERROR")

from hhop import DFExtender, SchemaManager, TablePartitionDescriber, SCD2Helper #main classes
from hhop import read_table, write_table, write_read_table, union_all, deduplicate_df, get_table_location # useful functions
from hhop import HhopException
display(spark)

23/07/15 17:21:01 WARN Utils: Your hostname, Pavels-MacBook-Air.local resolves to a loopback address: 127.0.0.200; using 192.168.0.103 instead (on interface en0)
23/07/15 17:21:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/15 17:21:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/15 17:21:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df1_transactions = spark.read.csv('../hhop/scd2_data/to_scd2.csv', sep=';', header=True)

In [3]:
df1_transactions.orderBy(['pk1', 'pk2', 'ts']).show(100, False)

+---+----+------+------+------+-----------+-------------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |
+---+----+------+------+------+-----------+-------------------+
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|
|v1 |null|null  |fds   |null  |null       |2023-05-11 15:00:00|
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|
|v1 |c1  |a1    |b1    |c1    |r1         |2023-05-01 10:00:00|
|v1 |c1  |a1    |b1    |c1    |r2         |2023-05-01 12:00:00|
|v1 |c1  |a1    |b1    |c1    |r3         |2023-05-02 12:00:00|
|v1 |c1  |a1    |b1    |c2    |null       |2023-05-03 12:00:00|
|v1 |c1  |a1    |b2    |c2    |null       |2023-05-03 15:00:00|
|v1 |c1  |null  |b2    |c2    |r3         |2023-05-05 15:00:00|
|v1 |c1  |null  |b2    |c2    |r3         |2023-05-06 15:00:00|
|v1 |c1  |null  |null  |c2    |r3         |2023-05-07 15:00:00|
|v1 |c1  |null  |null  |null  |null     

In [28]:
import hhop
reload(hhop)
reload(hhop.hhop)
reload(hhop.hhop.main)
from hhop import SCD2Helper

In [29]:
df1_transactions_s = SCD2Helper(
    df1_transactions, 
    pk=['pk1', 'pk2'], 
    non_pk=['nonpk1', 'nonpk2', 'nonpk3'],
    time_col='ts',
)

In [30]:
df1_scd2 = df1_transactions_s.df_to_scd2().cache()

In [31]:
df1_scd2.orderBy(['pk1', 'pk2', 'ts']).show(100, False)

+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |row_hash                        |row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|56e6807f4b745e20dffeb1b731e5a6d4|2023-05-07     |2023-05-09   |
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|6654c734ccab8f440ff0825eb443dc7f|2023-05-10     |2023-05-10   |
|v1 |null|null  |fds   |null  |null       |2023-05-11 15:00:00|2d2722576095dd7996570b307d777539|2023-05-11     |2023-05-11   |
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|b08363345cd7c1cb14e6f4747ce1563d|2023-05-12     |9999-12-31   |
|v1 |c1  |a1    |b1    |c1    |r1         |2023-05-01 10:00:00|93e6cc4b8b0445cf261e9417106ae6f0|2023-05-01     

In [38]:
type(df1_scd2)

hhop.hhop.main.SCD2Helper

In [46]:
df1_scd2.count()

10

In [55]:
df1_scd2.withColumnRenamed('row_actual_to', 'asdf').show(10)

+---+----+------+------+------+-----------+-------------------+--------------------+---------------+----------+
|pk1| pk2|nonpk1|nonpk2|nonpk3|nonpk_extra|                 ts|            row_hash|row_actual_from|      asdf|
+---+----+------+------+------+-----------+-------------------+--------------------+---------------+----------+
| v1|  c1|    a1|    b1|    c1|         r1|2023-05-01 10:00:00|93e6cc4b8b0445cf2...|     2023-05-01|2023-05-02|
| v1|  c1|    a1|    b2|    c2|       null|2023-05-03 15:00:00|a6244d3c7c2aed33c...|     2023-05-03|2023-05-04|
| v1|  c1|  null|    b2|    c2|         r3|2023-05-05 15:00:00|17f599be9e07976c2...|     2023-05-05|2023-05-06|
| v1|  c1|  null|  null|    c2|         r3|2023-05-07 15:00:00|a363a9dd6d5b30865...|     2023-05-07|2023-05-09|
| v1|  c1|  null|  null|  null|       null|2023-05-10 15:00:00|da58ea33b20d82042...|     2023-05-10|2023-05-12|
| v1|  c1|  null|  null|    c2|         r3|2023-05-13 15:00:00|a363a9dd6d5b30865...|     2023-05-13|9999

## Validation

In [7]:
s2 = s1h.df_to_scd2(s1, ['pk1', 'pk2'], ['nonpk1', 'nonpk2', 'nonpk3'], 'ts').orderBy(['pk1', 'pk2', 'ts']).cache()

In [8]:
import hhop
reload(hhop)
from hhop import SCD2Helper

In [9]:
s2.show(100, False)

+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |row_hash                        |row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|56e6807f4b745e20dffeb1b731e5a6d4|2023-05-07     |2023-05-09   |
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|6654c734ccab8f440ff0825eb443dc7f|2023-05-10     |2023-05-10   |
|v1 |null|null  |fds   |null  |null       |2023-05-11 15:00:00|2d2722576095dd7996570b307d777539|2023-05-11     |2023-05-11   |
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|b08363345cd7c1cb14e6f4747ce1563d|2023-05-12     |9999-12-31   |
|v1 |c1  |a1    |b1    |c1    |r1         |2023-05-01 10:00:00|93e6cc4b8b0445cf261e9417106ae6f0|2023-05-01     

In [10]:
# s3 = s2.filter("row_hash != 'da58ea33b20d82042d9969c46c16c3b8'")
s3 = s2.withColumn('row_actual_to', F.substring('row_actual_to', 1, 6))

In [11]:
# .df_to_scd2(['nonpk1', 'nonpk2', 'nonpk3'], 'ts').orderBy(['pk1', 'pk2', 'ts'])
a = SCD2Helper()
a.validate_scd2(s3, ['pk1', 'pk2'], ['nonpk1', 'nonpk2', 'nonpk3'], 'ts')

There are 2 PK duplicates by ['pk1', 'pk2', 'row_actual_to'] Look at `.basic_pk_check.df_duplicates_pk`
10 rows with invalid dates, look at `.df_invalid_dates`
Number of records: 10


In [12]:
# .df_to_scd2(['nonpk1', 'nonpk2', 'nonpk3'], 'ts').orderBy(['pk1', 'pk2', 'ts'])
a = SCD2Helper()
a.validate_scd2(s2, ['pk1', 'pk2'], ['nonpk1', 'nonpk2', 'nonpk3'], 'ts')

Number of records: 10
All tests passed


In [14]:
# a.basic_pk_check.df_duplicates_pk.show(10, False)

### Fill history

In [13]:
s2.show(100, False)

+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |row_hash                        |row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|56e6807f4b745e20dffeb1b731e5a6d4|2023-05-07     |2023-05-09   |
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|6654c734ccab8f440ff0825eb443dc7f|2023-05-10     |2023-05-10   |
|v1 |null|null  |fds   |null  |null       |2023-05-11 15:00:00|2d2722576095dd7996570b307d777539|2023-05-11     |2023-05-11   |
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|b08363345cd7c1cb14e6f4747ce1563d|2023-05-12     |9999-12-31   |
|v1 |c1  |a1    |b1    |c1    |r1         |2023-05-01 10:00:00|93e6cc4b8b0445cf261e9417106ae6f0|2023-05-01     

In [14]:
@F.udf(T.ArrayType(T.StructType()))
def merge_columns(col1, col2):
    merged = [x for x in col1] + [y for y in col2]
    return merged

# result_df = df.select([
#   "column3",
#   merge_columns("column1", "column2").alias("merged")
# ])


In [45]:
window_inc_versions = W.partitionBy('pk1', 'pk2').orderBy('row_actual_from')
s2_holes = s2.filter(
        ~col('row_hash').isin([
            'a6244d3c7c2aed33c4d9525fbef29c1d', 
            'da58ea33b20d82042d9969c46c16c3b8', '2d2722576095dd7996570b307d777539', 
                               'a363a9dd6d5b30865ab5813581941516'
                               ])
)

In [46]:
s2_holes.show()

+---+----+------+------+------+-----------+-------------------+--------------------+---------------+-------------+
|pk1| pk2|nonpk1|nonpk2|nonpk3|nonpk_extra|                 ts|            row_hash|row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------+---------------+-------------+
| v1|null|  null|  null|    c2|         r3|2023-05-07 15:00:00|56e6807f4b745e20d...|     2023-05-07|   2023-05-09|
| v1|null|  null|  null|  null|       null|2023-05-10 15:00:00|6654c734ccab8f440...|     2023-05-10|   2023-05-10|
| v1|null|  null|   fds|  asdf|       null|2023-05-12 15:00:00|b08363345cd7c1cb1...|     2023-05-12|   9999-12-31|
| v1|  c1|    a1|    b1|    c1|         r1|2023-05-01 10:00:00|93e6cc4b8b0445cf2...|     2023-05-01|   2023-05-02|
| v1|  c1|  null|    b2|    c2|         r3|2023-05-05 15:00:00|17f599be9e07976c2...|     2023-05-05|   2023-05-06|
+---+----+------+------+------+-----------+-------------------+-----------------

In [57]:

(
    s2_holes
    .withColumn('earliest_from_in_hole', F.coalesce(F.date_add(F.lag('row_actual_to').over(window_inc_versions), 1), F.lit('1000-01-01')))
    .withColumn('is_need_fill_behind', col('row_actual_from') != col('earliest_from_in_hole'))
    .withColumn('hole_behind', F.when(col('is_need_fill_behind'), F.concat_ws(',', col('earliest_from_in_hole'), F.date_sub('row_actual_from', 1))))
    .withColumn('hole_plus_infinity_history', F.when(
        F.lead('row_actual_to').over(window_inc_versions).isNull() & (col('row_actual_to') != F.lit('9999-12-31')),
          F.concat_ws(',', F.date_add('row_actual_to', 1), F.lit('9999-12-31'))
          )
        )       
        .withColumn('merged_history', F.concat_ws(';', 'hole_behind', 'hole_plus_infinity_history'))
        .withColumn('from_to_str', F.explode(F.split('merged_history', ';')))
        .filter(col('from_to_str') != '')
        .withColumn('splitted_from_to', F.split('from_to_str', ','))
        .withColumn('from', col('splitted_from_to').getItem(0))
        .withColumn('to', col('splitted_from_to').getItem(1))
    .drop('row_hash', 'ts', 'nonpk3')
).show(100, False)

+---+----+------+------+-----------+---------------+-------------+---------------------+-------------------+---------------------+--------------------------+-------------------------------------------+---------------------+------------------------+----------+----------+
|pk1|pk2 |nonpk1|nonpk2|nonpk_extra|row_actual_from|row_actual_to|earliest_from_in_hole|is_need_fill_behind|hole_behind          |hole_plus_infinity_history|merged_history                             |from_to_str          |splitted_from_to        |from      |to        |
+---+----+------+------+-----------+---------------+-------------+---------------------+-------------------+---------------------+--------------------------+-------------------------------------------+---------------------+------------------------+----------+----------+
|v1 |c1  |a1    |b1    |r1         |2023-05-01     |2023-05-02   |1000-01-01           |true               |1000-01-01,2023-04-30|null                      |1000-01-01,2023-04-30         

In [86]:
import hhop
reload(hhop)
from hhop import SCD2Helper

In [87]:
s = SCD2Helper(s2_holes, ['pk1', 'pk2'], [])

In [90]:
s.fill_scd2_history().orderBy('pk1', 'pk2', 'row_actual_from').show(10, False)

+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |row_hash                        |row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|v1 |null|null  |null  |null  |null       |null               |6654c734ccab8f440ff0825eb443dc7f|1000-01-01     |2023-05-06   |
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|56e6807f4b745e20dffeb1b731e5a6d4|2023-05-07     |2023-05-09   |
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|6654c734ccab8f440ff0825eb443dc7f|2023-05-10     |2023-05-10   |
|v1 |null|null  |null  |null  |null       |null               |6654c734ccab8f440ff0825eb443dc7f|2023-05-11     |2023-05-11   |
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|b08363345cd7c1cb14e6f4747ce1563d|2023-05-12     

In [91]:
k = s.fill_scd2_history()

In [94]:
k

pyspark.sql.dataframe.DataFrame

In [92]:
# SCD2Helper(k, ['pk1', 'pk2'], []).validate_scd2()

TypeError: SCD2Helper.validate_scd2() missing 4 required positional arguments: 'df', 'pk', 'non_pk', and 'time_col'

In [94]:
# s2_holes.show(10, False)

+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |row_hash                        |row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|56e6807f4b745e20dffeb1b731e5a6d4|2023-05-07     |2023-05-09   |
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|6654c734ccab8f440ff0825eb443dc7f|2023-05-10     |2023-05-10   |
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|b08363345cd7c1cb14e6f4747ce1563d|2023-05-12     |9999-12-31   |
|v1 |c1  |a1    |b1    |c1    |r1         |2023-05-01 10:00:00|93e6cc4b8b0445cf261e9417106ae6f0|2023-05-01     |2023-05-02   |
|v1 |c1  |null  |b2    |c2    |r3         |2023-05-05 15:00:00|17f599be9e07976c2036361c9ad8f633|2023-05-05     

In [111]:
import hhop
reload(hhop)
from hhop import SCD2Helper
(
    SCD2Helper(s2, ['pk1', 'pk2'], ['nonpk2']).merge_scd2_history()
).orderBy('pk1', 'pk2', 'row_actual_from').show(10)

+---+----+------+------+------+-----------+-------------------+--------------------+---------------+-------------+
|pk1| pk2|nonpk1|nonpk2|nonpk3|nonpk_extra|                 ts|            row_hash|row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------+---------------+-------------+
| v1|null|  null|  null|    c2|         r3|2023-05-07 15:00:00|6654c734ccab8f440...|     2023-05-07|   2023-05-10|
| v1|null|  null|   fds|  null|       null|2023-05-11 15:00:00|2d2722576095dd799...|     2023-05-11|   9999-12-31|
| v1|  c1|    a1|    b1|    c1|         r1|2023-05-01 10:00:00|096c3c37214aa93e8...|     2023-05-01|   2023-05-02|
| v1|  c1|    a1|    b2|    c2|       null|2023-05-03 15:00:00|e3a0efcb4f913e410...|     2023-05-03|   2023-05-06|
| v1|  c1|  null|  null|    c2|         r3|2023-05-07 15:00:00|da58ea33b20d82042...|     2023-05-07|   9999-12-31|
+---+----+------+------+------+-----------+-------------------+-----------------

In [101]:
import hhop
reload(hhop)
from hhop import SCD2Helper
(
    SCD2Helper(s2, ['pk1', 'pk2'], ['nonpk2']).merge_scd2_history()
).show(10)

+---+----+------+------+------+-----------+-------------------+--------------------+---------------+-------------+-----------+---------+
|pk1| pk2|nonpk1|nonpk2|nonpk3|nonpk_extra|                 ts|            row_hash|row_actual_from|row_actual_to|version_num|latest_to|
+---+----+------+------+------+-----------+-------------------+--------------------+---------------+-------------+-----------+---------+
| v1|  c1|    a1|    b1|    c1|         r1|2023-05-01 10:00:00|096c3c37214aa93e8...|     2023-05-01|   2023-05-02|          0|     null|
| v1|  c1|    a1|    b2|    c2|       null|2023-05-03 15:00:00|e3a0efcb4f913e410...|     2023-05-03|   2023-05-04|          1|     null|
| v1|  c1|  null|  null|    c2|         r3|2023-05-07 15:00:00|da58ea33b20d82042...|     2023-05-07|   2023-05-09|          2|     null|
| v1|null|  null|  null|    c2|         r3|2023-05-07 15:00:00|6654c734ccab8f440...|     2023-05-07|   2023-05-09|          0|     null|
| v1|null|  null|   fds|  null|       nul

In [98]:
s2_inf = (
    s2_holes
    .select('pk1', 'pk2')
    .distinct()
    .withColumn('row_actual_from', F.lit('1000-01-01'))
    .withColumn('row_actual_to', F.lit('9999-12-31'))
)

In [ ]:
(
    s2_inf
    .join(s2_holes, on=['pk1', 'pk2'], how='inner')
    .filter()
)

In [52]:
a = [('aaa', 'tom', 'in', 'nats', 'in'),('bbb', 'tom1', 'on', 'nats1', 'on'),]
df = spark.createDataFrame(a, ['id', 'f_name',  'f_add',  'l_name', 'l_add'])

+---+------+-----+------+-----+
| id|f_name|f_add|l_name|l_add|
+---+------+-----+------+-----+
|aaa|   tom|   in|  nats|   in|
|bbb|  tom1|   on| nats1|   on|
+---+------+-----+------+-----+



In [79]:
l = (
    df
    .withColumn('t1', F.struct('l_add', 'l_name'))
    .withColumn('t2', F.struct('f_add', 'f_name'))
    .groupBy('id')
    .agg(
        F.collect_list('t1').alias('s1'),
        F.collect_list('t2').alias('s2'),
        )
    # .select(F.concat('t1', 't2'))
)
l.show()

+---+-------------+------------+
| id|           s1|          s2|
+---+-------------+------------+
|aaa| [{in, nats}]| [{in, tom}]|
|bbb|[{on, nats1}]|[{on, tom1}]|
+---+-------------+------------+



In [80]:
l

DataFrame[id: string, s1: array<struct<l_add:string,l_name:string>>, s2: array<struct<f_add:string,f_name:string>>]

In [ ]:
def merge_history(df):

In [91]:
s2.show(100, False)

+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|pk1|pk2 |nonpk1|nonpk2|nonpk3|nonpk_extra|ts                 |row_hash                        |row_actual_from|row_actual_to|
+---+----+------+------+------+-----------+-------------------+--------------------------------+---------------+-------------+
|v1 |null|null  |null  |c2    |r3         |2023-05-07 15:00:00|56e6807f4b745e20dffeb1b731e5a6d4|2023-05-07     |2023-05-09   |
|v1 |null|null  |null  |null  |null       |2023-05-10 15:00:00|6654c734ccab8f440ff0825eb443dc7f|2023-05-10     |2023-05-10   |
|v1 |null|null  |fds   |null  |null       |2023-05-11 15:00:00|2d2722576095dd7996570b307d777539|2023-05-11     |2023-05-11   |
|v1 |null|null  |fds   |asdf  |null       |2023-05-12 15:00:00|b08363345cd7c1cb14e6f4747ce1563d|2023-05-12     |9999-12-31   |
|v1 |c1  |a1    |b1    |c1    |r1         |2023-05-01 10:00:00|93e6cc4b8b0445cf261e9417106ae6f0|2023-05-01     

In [115]:
import inspect

In [120]:
class E:
    def __init__(self, a1, a2, a3=5, a4=8):
        print(locals())
        pass

In [121]:
k = E(1,2)

{'self': <__main__.E object at 0x7ff3b2297850>, 'a1': 1, 'a2': 2, 'a3': 5, 'a4': 8}


In [114]:
k

In [5]:
df1, df2 = [spark.read.csv(f'scd2_data/df_scd2_join_{i}.csv', sep=';', header=True) for i in range(1, 3)]

In [6]:
df2.show(10, False)

+---+---+--------+-------------------+
|pk1|pk2|phone_id|ts                 |
+---+---+--------+-------------------+
|v1 |c1 |e1      |2023-04-01 10:00:00|
|v1 |c1 |e2      |2023-05-06 12:00:00|
|v1 |c1 |e3      |2023-05-12 12:00:00|
|v1 |c1 |e1      |2023-05-13 12:00:00|
|v1 |c2 |e1      |2023-04-01 10:00:00|
|v1 |c2 |e2      |2023-05-06 12:00:00|
|v1 |c2 |e3      |2023-05-12 12:00:00|
|v1 |c2 |e1      |2023-05-13 12:00:00|
+---+---+--------+-------------------+



In [7]:
# df1_scd2, df2_scd2 = [df_to_scd2(df, ['pk1', 'pk2'], [non_pk_col], 'ts').orderBy(['pk1', 'pk2', 'ts']).drop('ts') for df, non_pk_col in zip((df1, df2), ('email_id', 'phone_id'))]

In [8]:
df2_scd2.show(100, False)

+---+---+--------+--------------------------------+---------------+-------------+
|pk1|pk2|phone_id|row_hash                        |row_actual_from|row_actual_to|
+---+---+--------+--------------------------------+---------------+-------------+
|v1 |c1 |e1      |e14f0e80db49cd1501de87adf05f6022|2023-04-01     |2023-05-05   |
|v1 |c1 |e2      |9862c1fb9265b03695dc9a727406c43e|2023-05-06     |2023-05-11   |
|v1 |c1 |e3      |543b4e1fe15d3cd37fc7b9454156f4e1|2023-05-12     |2023-05-12   |
|v1 |c1 |e1      |e14f0e80db49cd1501de87adf05f6022|2023-05-13     |9999-12-31   |
|v1 |c2 |e1      |db078b8d7b629e8c3e11aeaf24952480|2023-04-01     |2023-05-05   |
|v1 |c2 |e2      |284ed4afc0045d818e840896714656ca|2023-05-06     |2023-05-11   |
|v1 |c2 |e3      |87795052bb06129a6007a0dfaad2efef|2023-05-12     |2023-05-12   |
|v1 |c2 |e1      |db078b8d7b629e8c3e11aeaf24952480|2023-05-13     |9999-12-31   |
+---+---+--------+--------------------------------+---------------+-------------+



In [9]:
def validate_df_scd2(df, pk, ): pass

In [10]:
def scd2_join(df1, df2, pk):
    df1, df2 = df1.alias('df1'), df2.alias('df2')
    
    tech_attr = {'row_actual_from', 'row_actual_to', 'row_hash'}
    def get_non_pk_attrs(df):
        all_attrs = set(df.columns)
        pk_attrs = set(pk)
        non_pk_attrs = all_attrs - tech_attr - pk_attrs
        return non_pk_attrs
    
    greatest_from = F.greatest(df1['row_actual_from'], df2['row_actual_from'])
    least_to = F.least(df1['row_actual_to'], df2['row_actual_to'])
    pk_cond_join = ' and '.join([f'df1.{pk_col} = df2.{pk_col}' for pk_col in pk])
    
    cond_scd2_join = F.expr(pk_cond_join) & (greatest_from <= least_to)
    df_joined = (
        df1
        .join(df2, on=cond_scd2_join, how='inner')
    )
    
    df_new_scd2 = (
        df_joined
        .select(
            *[f'df1.{pk_col}' for pk_col in pk], # из-за условия джоина нельзя просто так взять атрибуты из pk
            *get_non_pk_attrs(df1),
            *get_non_pk_attrs(df2),
            greatest_from.alias('row_actual_from'),
            least_to.alias('row_actual_to'),
        )
    )

    return df_new_scd2
scd2_join(df1_scd2, df2_scd2, ['pk1','pk2']).orderBy('pk1', 'pk2','row_actual_from').show(100, False)

+---+---+--------+--------+---------------+-------------+
|pk1|pk2|email_id|phone_id|row_actual_from|row_actual_to|
+---+---+--------+--------+---------------+-------------+
|v1 |c1 |e1      |e1      |2023-05-01     |2023-05-03   |
|v1 |c1 |e2      |e1      |2023-05-04     |2023-05-05   |
|v1 |c1 |e2      |e2      |2023-05-06     |2023-05-09   |
|v1 |c1 |e3      |e2      |2023-05-10     |2023-05-11   |
|v1 |c1 |e1      |e3      |2023-05-12     |2023-05-12   |
|v1 |c1 |e1      |e1      |2023-05-13     |9999-12-31   |
|v1 |c2 |e1      |e1      |2023-05-01     |2023-05-05   |
|v1 |c2 |e1      |e2      |2023-05-06     |2023-05-11   |
|v1 |c2 |e1      |e3      |2023-05-12     |2023-05-12   |
|v1 |c2 |e1      |e1      |2023-05-13     |9999-12-31   |
+---+---+--------+--------+---------------+-------------+



In [11]:
spark.stop()